# Generators And Coroutines #

Python 3.5 added *coroutine* functions defined with `async def`, along with other [related `async` constructs](https://docs.python.org/3/reference/compound_stmts.html#coroutines). The easiest way to make use of these features is in conjunction with an *event loop*, such as that provided by the standard [`asyncio`](https://docs.python.org/3/library/asyncio.html) library, or some other library that provides an `asyncio`-compatible API.

But you can also build your own custom event loop, by doing direct [`send()`](https://docs.python.org/3/reference/datamodel.html#coroutine-objects) calls to coroutine objects. `asyncio`, after all, is written in pure Python, so anything it can do, you can do in your own Python code. If you are curious about the nitty-gritty of such low-level event dispatching, read on.

First of all, let us review some aspects of Python that you may already familiar with, namely the concepts of iterators and generators. Then we will see how coroutines are a natural evolution of this line of concepts.

### Iterators ###

In Python, an *iterator* is an object that defines a [`__next__()`](https://docs.python.org/3/library/stdtypes.html#iterator.__next__) method. This allows it to be passed to the [`next()`](https://docs.python.org/3/library/functions.html#next) built-in function to obtain some sequence of elements in turn.

An iterator can be constructed from a Python sequence object using the [`iter()`](https://docs.python.org/3/library/functions.html#iter) built-in function, which looks for an `__iter__()` method on its argument. An object that defines such a method is called an *iterable*:

In [ ]:
some_list = ["a", "b", "c"]

each_elt = iter(some_list)
print(next(each_elt))
print(next(each_elt))
print(next(each_elt))
print(next(each_elt))

Notice the convention for raising `StopIteration` when the iterator runs out of results to return. As an alternative to the exception, the `next()` function has an option to return a special *sentinel* value in this case:

In [ ]:
some_list = ["a"]

each_elt = iter(some_list)
print(next(each_elt, "no more"))
print(next(each_elt, "no more"))

Of course, you have to choose the sentinel to be something that cannot possibly be mistaken for a valid element of the sequence. It is often possible to use `None` for this, but the choice is up to you.

An iterator can also be used in a `for`-statement, which will automatically catch the `StopIteration` exception and terminate:

In [ ]:
some_list = ["a", "b", "c"]

each_elt = iter(some_list)

for elt in each_elt :
    print(elt)
#end for

As a convenience, you don’t need to construct the iterator from the sequence object yourself: the `for`-statement is quite capable of doing it itself:

In [ ]:
for elt in some_list :
    print(elt)
#end for

It is easy enough to define your own iterator objects, for example:

In [ ]:
class FiboIterator :
    "the Fibonacci sequence, up to some specified limit."

    def __init__(self, limit) :
        self.limit = limit
        self.a, self.b = 0, 1
        self.count = 0
    #end __init__

    def __iter__(self) :
        return self
    #end __iter__

    def __next__(self) :
        if self.count >= self.limit :
            raise StopIteration
        #end if
        self.a, self.b = self.b, self.a + self.b
        self.count += 1
        return self.a
    #end __next__

#end FiboIterator

for i in FiboIterator(8) :
    print(i)
#end for

Here the iterable object and the iterator object are one and the same.

But in many cases it is easier to simply define a *generator function*.

### Generators ###

A *generator* is a function that behaves as an iterator: it does this by using the [`yield`](https://docs.python.org/3/reference/simple_stmts.html#the-yield-statement) construct instead of `return`. Whereas `return` terminates execution of the function, `yield` only *suspends* it, allowing the caller to resume from that point by calling its `__next__()` method. When the function does finally return, Python automatically raises a `StopIteration` exception on its behalf.

For example, the above Fibonacci iterator could have been written more simply as:

In [ ]:
def fibo_iterator(limit) :
    "the Fibonacci sequence, up to some specified limit."
    a, b = 0, 1
    for i in range(limit) :
        a, b = b, a + b
        yield a
    #end for
#end fibo_iterator

for i in fibo_iterator(8) :
    print(i)
#end for

Generators are like a cut-down version of an old computing concept known as *coroutines*. Python elaborated the generator concept over time, turning `yield` into an expression instead of a statement (allowing values to be passed from the caller to the coroutine via `send()`, rather than just having them go one way via `__next__()`), and also adding `yield from` to allow one generator to “delegate” part or all of its yield sequence to another iterator. These were clearly all attempts to get closer to the original, quite simple, coroutine concept, but at the same time they just added more complications to the language.

Finally, in Python 3.5, the Python designers realized that the only way out of this mess was to add *proper* coroutines.

### Coroutines ###

In Python, a *coroutine* is a function defined with `async def` instead of plain old `def`. Calling such a function does not immediately cause it to start execution: instead, the call returns a *coroutine object*, which is an example of an [*awaitable*](https://docs.python.org/3/reference/expressions.html#await-expression) object. Within a coroutine, it is possible to use the `await` construct to pass control to another awaitable, such as another coroutine object. It is also possible to define your own classes with an `__await__()` method that is a generator, and use those as awaitables.

But the `await` construct is not legal outside a coroutine. So how does your Python mainline start a coroutine executing in the first place? The answer is to call its `send()` method, just like was added to generators when `yield` was changed from a statement to an expression.

Here is a trivial example of coroutine invocation and use of `await` on another coroutine (notice how it raises `StopIteration` on falling off the end of `coro1`):

In [ ]:
async def coro1() :
    print("enter coro1")
    val = await coro2()
    print("got " + repr(val) + " from coro2")
    print("done coro1")
#end coro1

async def coro2() :
    return "hi from coro2"
#end coro2

coro = coro1()
coro.send(None)
  # first or only send() call must always pass None


That doesn’t look very useful, though, does it? The mainline is unable to do anything until both coroutines have finished execution. How would you implement an event loop like this? Because the whole point of such a loop is the ability to manage *multiple concurrent coroutines*, letting each one have some time until it voluntarily suspends itself, returning control to the dispatcher to allow it to give time to the next one.

But remember, *a class with an* `__await__()` *generator is also an awaitable*. And a generator can do a `yield` (which a coroutine itself cannot). When it does so, something magical happens: the coroutine remains suspended, but control returns to the mainline!

In [ ]:
async def coro3() :
    print("enter coro3")
    val = await Yielder("from coro3 to yielder")
    print("got " + repr(val) + " from yielder")
    print("done coro3")
#end coro3

class Yielder :

    def __init__(self, arg) :
        self.arg = arg
    #end __init

    def __await__(self) :
        print("enter Yielder.__await__")
        result = yield self.arg
        print("leaving Yielder.__await__")
        return result
    #end __await__

#end Yielder

coro = coro3()
print("about to send")
msg = coro.send(None)
print("back to mainline, got " + repr(msg))
coro.send("hi from mainline")

Notice the sequence of actions:

* The `coro3()` call creates the coroutine object but does not start it executing.
* The `send(None)` to the coroutine object actually starts `coro3` executing.
* `coro3` passes control to `Yielder.__await__()`.
* `Yielder.__await__()` passes control back to the mainline, returning the value passed when the `Yielder` object was created and awaited.
* The mainline does the second `send()` to the coroutine object, this time passing it a non-`None` value.
* `Yielder.__await__()` resumes execution and returns the value passed from the mainline back to its caller.
* `coro3` prints the value obtained from the `await` expression.
* `coro3` finishes execution, causing the `StopIteration` exception.

Why does a coroutine raise `StopIteration` when it finishes? Because that is the way it tells the event-loop dispatcher that it has finished doing what it has to do, and should not be scheduled for execution any more.

The permissible transfers of control between mainline code, coroutines and generators can be summed up in a diagram I call **Van Rossum’s Triangle**:

In [ ]:
%%svg
<?xml version="1.0" encoding="UTF-8"?>
<svg id="svg6" width="425pt" height="325pt" version="1.1" viewBox="0 0 1020 780" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
 <defs id="defs8">
  <marker id="marker5590" overflow="visible" orient="auto">
   <path id="path5588" transform="matrix(.4 0 0 .4 4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill="#9baa2d" fill-rule="evenodd" stroke="#9baa2d" stroke-width="1pt"/>
  </marker>
  <marker id="marker4768" overflow="visible" orient="auto">
   <path id="path4766" transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill="#9baa2d" fill-rule="evenodd" stroke="#9baa2d" stroke-width="1pt"/>
  </marker>
  <marker id="marker5770" overflow="visible" orient="auto">
   <path id="path5768" transform="matrix(.4 0 0 .4 4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill="#9baa2d" fill-rule="evenodd" stroke="#9baa2d" stroke-width="1pt"/>
  </marker>
  <marker id="marker4714" overflow="visible" orient="auto">
   <path id="path4712" transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill="#9baa2d" fill-rule="evenodd" stroke="#9baa2d" stroke-width="1pt"/>
  </marker>
  <marker id="marker11897" overflow="visible" orient="auto">
   <path id="path11895" transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill="#9baa2d" fill-rule="evenodd" stroke="#9baa2d" stroke-width="1pt"/>
  </marker>
  <marker id="marker11719" overflow="visible" orient="auto">
   <path id="path11717" transform="matrix(.4 0 0 .4 4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill="#9baa2d" fill-rule="evenodd" stroke="#9baa2d" stroke-width="1pt"/>
  </marker>
  <marker id="marker8315" overflow="visible" orient="auto">
   <path id="path8313" transform="matrix(.4 0 0 .4 4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill="#9baa2d" fill-rule="evenodd" stroke="#9baa2d" stroke-width="1pt"/>
  </marker>
  <marker id="marker6787" overflow="visible" orient="auto">
   <path id="path6785" transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill="#9baa2d" fill-rule="evenodd" stroke="#9baa2d" stroke-width="1pt"/>
  </marker>
  <marker id="Arrow1Mstart" overflow="visible" orient="auto">
   <path id="path4541" transform="matrix(.4 0 0 .4 4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill="#9baa2d" fill-rule="evenodd" stroke="#9baa2d" stroke-width="1pt"/>
  </marker>
  <marker id="Arrow1Mend" overflow="visible" orient="auto">
   <path id="path4544" transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill="#9baa2d" fill-rule="evenodd" stroke="#9baa2d" stroke-width="1pt"/>
  </marker>
 </defs>
 <metadata id="metadata1">
  <rdf:RDF>
   <cc:Work rdf:About="">
    <dc:format>image/svg+xml</dc:format>
    <dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
    <dc:title>Coroutines Versus Generators</dc:title>
   </cc:Work>
  </rdf:RDF>
 </metadata>
 <title id="title4">Coroutines Versus Generators</title>
 <text id="text4487" x="71.637589" y="425.29413" fill="#000000" font-family="'Linux Biolinum O'" font-size="48px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4485" x="71.637589" y="425.29413" font-family="'Linux Biolinum O'" font-size="48px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Mainline</tspan></text>
 <text id="text4491" x="416.25128" y="649.13202" fill="#000000" font-family="'Linux Biolinum O'" font-size="48px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4489" x="416.25128" y="649.13202" font-family="'Linux Biolinum O'" font-size="48px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Generator</tspan></text>
 <text id="text4495" x="406.5892" y="228.02695" fill="#000000" font-family="'Linux Biolinum O'" font-size="48px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4493" x="406.5892" y="228.02695" font-family="'Linux Biolinum O'" font-size="48px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Coroutine</tspan></text>
 <text id="text4499" x="213.34789" y="577.47174" fill="#2f2ce5" font-family="'Linux Biolinum O'" font-size="24px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4497" x="213.34789" y="577.47174" fill="#2f2ce5" font-family="'Linux Biolinum O'" font-size="24px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Send</tspan></text>
 <text id="text4503" x="218.17892" y="270.70111" fill="#2f2ce5" font-family="'Linux Biolinum O'" font-size="24px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4501" x="218.17892" y="270.70111" fill="#2f2ce5" font-family="'Linux Biolinum O'" font-size="24px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Send</tspan></text>
 <text id="text4507" x="289.83926" y="513.86304" fill="#7a4e1a" font-family="'Linux Biolinum O'" font-size="24px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4505" x="289.83926" y="513.86304" fill="#7a4e1a" font-family="'Linux Biolinum O'" font-size="24px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Yield</tspan></text>
 <text id="text4511" x="295.47546" y="325.45279" fill="#b70976" font-family="'Linux Biolinum O'" font-size="24px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4509" x="295.47546" y="325.45279" fill="#b70976" font-family="'Linux Biolinum O'" font-size="24px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Return</tspan></text>
 <text id="text4515" x="536.22192" y="447.03378" fill="#b70976" font-family="'Linux Biolinum O'" font-size="24px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4513" x="536.22192" y="447.03378" fill="#b70976" font-family="'Linux Biolinum O'" font-size="24px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Return</tspan></text>
 <text id="text4519" x="418.66681" y="447.03378" fill="#167c4a" font-family="'Linux Biolinum O'" font-size="24px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4517" x="418.66681" y="447.03378" fill="#167c4a" font-family="'Linux Biolinum O'" font-size="24px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Await</tspan></text>
 <text id="text4527" x="660.45715" y="185.60948" fill="#b70976" font-family="'Linux Biolinum O'" font-size="24px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4525" x="660.45715" y="185.60948" fill="#b70976" font-family="'Linux Biolinum O'" font-size="24px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Return</tspan></text>
 <text id="text4531" x="660.42004" y="261.97345" fill="#167c4a" font-family="'Linux Biolinum O'" font-size="24px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4529" x="660.42004" y="261.97345" fill="#167c4a" font-family="'Linux Biolinum O'" font-size="24px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Await</tspan></text>
 <path id="path4533" d="m180.34 351.22 183.58-136.07" fill="none" marker-end="url(#Arrow1Mend)" stroke="#9baa2d" stroke-width="3"/>
 <path id="path4947" d="m195.63 370.54 183.58-136.07" fill="none" marker-start="url(#Arrow1Mstart)" stroke="#9baa2d" stroke-width="3"/>
 <path id="path5297" d="m491.94 602.43-1e-5 -337.37" fill="none" marker-start="url(#marker11719)" stroke="#9baa2d" stroke-width="3"/>
 <path id="path6503" d="m518.51 599.21-1e-5 -334.15" fill="none" marker-end="url(#marker11897)" stroke="#9baa2d" stroke-width="3"/>
 <path id="path6783" d="m381.63 608.87-185.99-151.37" fill="none" marker-end="url(#marker6787)" stroke="#9baa2d" stroke-width="3"/>
 <path id="path8061" d="m366.33 627.39-185.99-151.37" fill="none" marker-start="url(#marker8315)" stroke="#9baa2d" stroke-width="3"/>
 <text id="text11235" x="760.49969" y="228.02695" fill="#000000" font-family="'Linux Biolinum O'" font-size="48px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan11233" x="760.49969" y="228.02695" font-family="'Linux Biolinum O'" font-size="48px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Coroutine</tspan></text>
 <text id="text4620" x="285.94885" y="750.99884" fill="#000000" font-family="'Linux Biolinum O'" font-size="24px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4618" x="285.94885" y="750.99884" font-family="'Linux Biolinum O'" font-size="24px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Generators and coroutines can also be mainlines</tspan></text>
 <text id="text4702" x="760.49969" y="649.13202" fill="#000000" font-family="'Linux Biolinum O'" font-size="48px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan4700" x="760.49969" y="649.13202" font-family="'Linux Biolinum O'" font-size="48px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Generator</tspan></text>
 <path id="path4710" d="m746.86 630.12-108.96-0.069" fill="none" marker-end="url(#marker4714)" stroke="#9baa2d" stroke-width="3"/>
 <text id="text5528" x="669.78754" y="609.9126" fill="#7a4e1a" font-family="'Linux Biolinum O'" font-size="24px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan5526" x="669.78754" y="609.9126" fill="#7a4e1a" font-family="'Linux Biolinum O'" font-size="24px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Yield</tspan></text>
 <text id="text5532" x="641.73761" y="682.16223" fill="#7a4e1a" font-family="'Linux Biolinum O'" font-size="24px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan5530" x="641.73761" y="682.16223" fill="#7a4e1a" font-family="'Linux Biolinum O'" font-size="24px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Yield From</tspan></text>
 <path id="path5540" d="m745.66 649.67-107.12-0.069" fill="none" marker-start="url(#marker5770)" stroke="#9baa2d" stroke-width="3"/>
 <text id="text6727" x="762.79669" y="675.34918" fill="#000000" font-family="'Linux Biolinum O'" font-size="24px" letter-spacing="0px" stroke-width=".75px" word-spacing="0px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal;line-height:18.75px" xml:space="preserve"><tspan id="tspan6725" x="762.79669" y="675.34918" font-family="'Linux Biolinum O'" font-size="24px" stroke-width=".75px" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Actually any iterator</tspan></text>
 <path id="path4666" d="m746.86 209.28-108.96-0.069" fill="none" marker-end="url(#marker4768)" stroke="#9baa2d" stroke-width="3"/>
 <path id="path4668" d="m745.66 228.83-107.12-0.069" fill="none" marker-start="url(#marker5590)" stroke="#9baa2d" stroke-width="3"/>
</svg>


### Why Coroutines? ###

Python already supports [threading](https://docs.python.org/3/library/threading.html), which is a well-established technique for doing in-process concurrency. Multiple threads can share the same process context, which means that they can, in principle, communicate with more direct, lower-overhead techniques than separate processes would require, with little or no need for intervention by the kernel.

But the downside of this is that **threads are notoriously hard to get right**. The *default-shared-everything* model means that multiple threads might inadvertently be accessing common objects without proper synchronization, and finding and fixing such bugs can be very tricky, since they are invariably timing-dependent, and even something simple as trying to insert debug messages might throw off the timing enough to stop the bug from happening.

So, even though threads can, in principle, do everything coroutines can do and more, nevertheless, there is still a place for the less-ambitious, less-troublesome coroutine concept.

Basically, the main situation where you *cannot* avoid threading is when you are performing some CPU-intensive task that can benefit from running on multiple CPUs in parallel. If your problem is not CPU-bound but is I/O-bound, then threads don’t buy you anything, and you might as well use coroutines.


### Implementing An Event Loop ###

Let us consider, step-by-step, the process for creating a usable event loop. To work properly with other `asyncio`-compatible Python code, this should be a subclass of [`AbstractEventLoop`](https://docs.python.org/3/library/asyncio-eventloop.html#asyncio.AbstractEventLoop); we’ll ignore that detail for now, but I will try to follow naming conventions suitable for such a subclass.

It is common for an event loop to provide a `Future` class: an object of this type is like an empty box with a sign above it saying “WATCH THIS SPACE”: one or more tasks can block themselves on a `Future`, waiting for something to appear in the box, while at some point some other task will indeed put something in, at which point the waiting tasks can resume execution.

An event loop will probably also want to define a wrapper for its schedulable entities: we will call this a `Task`. So the outline of our event-loop class might look like:

    class CustomEventLoop :

        class Future :
            "something that a task can block on by awaiting, to be" \
            " resumed when another task puts a result into it."

            def __init__(self, loop) :
                self.loop = loop
                self.awaiting = []
                self.done = False
                self.result = None
            #end __init__

            def __await__(self) :
                if not self.done :
                    self.awaiting.append(self.loop.cur)
                    yield None # value yielded doesn’t matter
                #end if
                assert self.done
                return \
                    self.result
            #end __await__

            def set_result(self, result) :
                "awakes all tasks awaiting this Future."
                assert not self.done
                self.result = result
                self.done = True
                for awaiting in self.awaiting :
                    self.loop.runnable.append(awaiting)
                #end for
                self.awaiting = None
            #end set_result

        #end Future

        class Task :
            "schedulable wrapper for coroutine objects."

            def __init__(self, loop, coro) :
                self.coro = coro
                self.is_alive = True
                self.done = loop.create_future()
            #end __init__

            def __await__(self) :
                return \
                    self.done.__await__()
            #end __await__

            ... # more to come

        #end Task

        def __init__(self) :
            self.runnable = []
            self.cur = None
            self.running = True
        #end __init__

        def create_future(self) :
            "creates a Future which can be awaited until" \
            " it has a result."
            return \
                self.Future(self)
        #end create_future

        ... # more to come

    #end CustomEventLoop

Notice the following:
* a `CustomEventLoop` instance has a `runnable` queue of `Task` objects and a `cur` which points to the currently-running `Task` (handy if the coroutine makes a call to a routine which needs to refer to the current task, such as `Future.__await__()`).
* a `Future` keeps a reference to the owning event loop, and also an `awaiting` queue of tasks that are waiting for the future to acquire a value.
* If the future doesn’t have a value yet, then `Future.__await__()` places the current task (in other words, the one making the `await` call) on its `awaiting` queue, then does a `yield` to return control back to the event loop mainline.
* `Future.set_result()` wakes up all tasks waiting for this future to acquire a value by putting them onto the `runnable` queue of the parent event loop.
* as a convenience, a `Task` automatically creates a `Future` for itself, that other tasks can block on to await the completion of this task. It references this in its own `__await__()` method, which makes a task an awaitable object.

**Reference circularity:** the `CustomEventLoop` has a `runnable` list of `Task`s, while each `Task` contains a reference to a `Future`, which in turn contains a reference back to the `CustomEventLoop`. This kind of thing stops Python from using reference-counting to free up storage, forcing it to resort to full-on [garbage collection](https://docs.python.org/3/library/gc.html). If this bothers you, the obvious way to fix it would be for a `Future` to keep only a [weak](https://docs.python.org/3/library/weakref.html#weakref.ref) reference to its parent `CustomEventLoop`. I’ll leave the details of this as an exercise for the reader.

How does the loop actually process tasks? The task dispatch routine might look like this:

    class CustomEventLoop :

        ... # as before ...

        def run_forever(self) :
            while self.running :
                try :
                    self.cur = self.runnable.pop(0)
                except IndexError :
                    raise RuntimeError("no more tasks")
                #end try
                self.cur.resume()
                self.cur = None
            #end while
        #end run_forever

and we might provide a `stop()` method to allow any task to terminate the loop like this:

        def stop(self) :
            self.running = False
        #end stop

        ... # as before ...

    #end CustomEventLoop

And what does `Task.resume()` do? That could be written this way:

        class Task :

        ... # as before ...

            def resume(self) :
                try :
                    self.coro.send(None)
                except StopIteration :
                    self.is_alive = False
                    self.done.set_result(None)
                #end try
            #end resume

        ... # as before ...

    #end Task

Note that the `Task` does not put itself back on the `CustomEventLoop.runnable` queue if the coroutine has not finished yet; that is the job of `Future.__await__()`.

So how do we actually put a coroutine on the event loop in the first place? The `CustomEventLoop.create_task()` routine might look like this:

    class CustomEventLoop :

        ... # as before ...

        def create_task(self, coro) :
            "puts a coroutine object on the execution queue."
            result = self.Task(self, coro)
            self.runnable.append(result)
            return \
                result
        #end create_task

        ... # as before ...

    #end CustomEventLoop

Let us put all this code together:

In [ ]:
class CustomEventLoop :
    "just about the most basic possible event loop."
    
    class Future :
        "something that a task can block on by awaiting, to be" \
        " resumed when another task puts a result into it."

        def __init__(self, loop) :
            self.loop = loop
            self.awaiting = []
            self.done = False
            self.result = None
        #end __init__

        def __await__(self) :
            if not self.done :
                self.awaiting.append(self.loop.cur)
                yield None # value yielded doesn’t matter
            #end if
            assert self.done
            return \
                self.result
        #end __await__

        def set_result(self, result) :
            "awakes all tasks awaiting this Future."
            assert not self.done
            self.done = True
            self.result = result
            for awaiting in self.awaiting :
                self.loop.runnable.append(awaiting)
            #end for
            self.awaiting = None
        #end set_result

    #end Future

    class Task :
        "schedulable wrapper for coroutine objects."

        def __init__(self, loop, coro) :
            self.coro = coro
            self.is_alive = True
            self.done = loop.create_future()
        #end __init__

        def __await__(self) :
            return \
                self.done.__await__()
        #end __await__

        def resume(self) :
            try :
                self.coro.send(None)
            except StopIteration :
                self.is_alive = False
                self.done.set_result(None)
            #end try
        #end resume

    #end Task

    def __init__(self) :
        self.runnable = []
        self.cur = None
        self.running = True
    #end __init__

    def create_future(self) :
        "creates a Future which can be awaited until" \
        " it has a result."
        return \
            self.Future(self)
    #end create_future

    def create_task(self, coro) :
        "puts a coroutine object on the execution queue."
        result = self.Task(self, coro)
        self.runnable.append(result)
        return \
            result
    #end create_task

    def run_forever(self) :
        while self.running :
            try :
                self.cur = self.runnable.pop(0)
            except IndexError :
                raise RuntimeError("no more tasks")
            #end try
            self.cur.resume()
            self.cur = None
        #end while
    #end run_forever

    def stop(self) :
        self.running = False
    #end stop

#end CustomEventLoop

And try making use of it:

In [ ]:
loop = CustomEventLoop()
fute1 = loop.create_future()
fute2 = loop.create_future()

async def task1() :
    print("task1 starting")
    val = await fute1
    print("fute1 result =", val)
    fute2.set_result(val)
    print("task1 finishing")
#end task1

async def task2() :
    print("task2 starting")
    val = await fute2
    print("task2 finishing with", val)
#end task2

async def task3() :
    print("task3 starting")
    val = await fute2
    print("task3 finishing with", val)
    loop.stop()
#end task3

loop.create_task(task1())
loop.create_task(task2())
loop.create_task(task3())
fute1.set_result("to fute1")
loop.run_forever()

This is fine, but it’s all very bare-bones. One common need is to be able to delay execution of a task for a specified interval. We cannot have individual tasks directly calling [`time.sleep()`](https://docs.python.org/3/library/time.html#time.sleep) for this purpose, because that will block the entire thread, and hence the entire event loop. So the event loop itself needs to provide a sleep function that only blocks the calling task.

Let us start by implementing [`call_later()` and `call_at()`](https://docs.python.org/3/library/asyncio-eventloop.html#delayed-calls) methods in our event loop. These invoke ordinary functions as callbacks at the appointed time; later we will build on these to implement synchronization of coroutine tasks.

The obvious thing to do is keep our queue of pending timers in an array, sorted by due time. We can conveniently use the standard [`heapq`](https://docs.python.org/3/library/heapq.html) module to keep the array efficiently sorted:

    import heapq

What does an array entry look like? Let us define a `Timer` class for this purpose, as follows:

    class Timer :
        "for keeping track of timed callbacks."

        def __init__(self, loop, when, action, *actionargs) :
            self.when = when
            self.loop = loop
            heapq.heappush(self.loop.timers, self)
            self.action = action
            self.actionargs = actionargs
        #end __init__

defining an `__lt__` method is sufficient for `heapq` to keep the entries sorted by increasing due time:

        def __lt__(a, b) :
            return \
                a.when < b.when
        #end __lt__

    #end Timer

where the parent `CustomEventLoop` will need its `__init__()` method augmented with the line:

        self.timers = []

We will also need a [time source](https://docs.python.org/3/library/time.html):

    import time

So that `CustomEventLoop` can now define its own idea of time:

    def time(self) :
        return \
            time.monotonic()
    #end time

(Don’t worry about the seeming inconsistency in the reference to the name “`time`”: this will work fine when the above definition is placed inside the `CustomEventLoop` class.)

Defining `CustomEventLoop.call_at()` now becomes quite simple:

    def call_at(self, when, action, *actionargs) :
        return \
            self.Timer(self, when, action, actionargs)
    #end call_at

where `when` defines the time at which to invoke the callback, referenced to `CustomEventLoop.time()`. Implementing an interval delay can be done just as easily:

    def call_later(self, delay, action, *actionargs) :
        return \
            self.call_at(self.time() + delay, action, *actionargs)
    #end call_later

The task dispatching routine now needs a bit more work. We continue processing the `runnable` queue as before, but when that empties, we now need to check the `timers` to see if anything needs to be done there. If the earliest timer is not due yet, we can call `time.sleep()` (which blocks the event loop, but then it has nothing else to do anyway) until the appropriate time.


    def run_forever(self) :
        while self.running :
            try :
                self.cur = self.runnable.pop(0)
            except IndexError :
                self.cur = None
            #end try
            if self.cur != None :
                self.cur.resume()
                self.cur = None
            else :
                if len(self.timers) != 0 :
                    delay = self.timers[0].when - self.time()
                    if delay > 0 :
                        time.sleep(delay)
                        # event loop has nothing else to do
                    #end if
                    entry = heapq.heappop(self.timers)
                    entry.action(*entry.actionargs)
                else :
                    raise RuntimeError("no more tasks")
                #end if
            #end if
        #end while
    #end run_forever

**Should runnable tasks take priority over timer expiry?** The above defers processing of expired timers as long as the `runnable` queue is not empty; should the priority be the other way round? Namely, as long as at least one timer has expired, that should be processed first, ahead of the runnable queue? This is a scheduling policy decision: how would the code look the other way? Perhaps you want to give the caller a way to control these priorities?

And here is what the complete `CustomEventLoop` class now looks like:

In [ ]:
import time
import heapq

class CustomEventLoop :
    "event loop with timer callbacks."
    
    class Future :
        "something that a task can block on by awaiting, to be" \
        " resumed when another task puts a result into it."

        def __init__(self, loop) :
            self.loop = loop
            self.awaiting = []
            self.done = False
            self.result = None
        #end __init__

        def __await__(self) :
            if not self.done :
                self.awaiting.append(self.loop.cur)
                yield None # value yielded doesn’t matter
            #end if
            assert self.done
            return \
                self.result
        #end __await__

        def set_result(self, result) :
            "awakes all tasks awaiting this Future."
            assert not self.done
            self.done = True
            self.result = result
            for awaiting in self.awaiting :
                self.loop.runnable.append(awaiting)
            #end for
            self.awaiting = None
        #end set_result

    #end Future

    class Task :
        "schedulable wrapper for coroutine objects."

        def __init__(self, loop, coro) :
            self.coro = coro
            self.is_alive = True
            self.done = loop.create_future()
        #end __init__

        def __await__(self) :
            return \
                self.done.__await__()
        #end __await__

        def resume(self) :
            try :
                self.coro.send(None)
            except StopIteration :
                self.is_alive = False
                self.done.set_result(None)
            #end try
        #end resume

    #end Task

    class Timer :
        "for keeping track of timed callbacks."

        def __init__(self, loop, when, action, actionargs) :
            self.when = when
            self.loop = loop
            heapq.heappush(self.loop.timers, self)
            self.action = action
            self.actionargs = actionargs
        #end __init__

        def __lt__(a, b) :
            "defines enough ordering for heapq."
            return \
                a.when < b.when
        #end __lt__

    #end Timer
    
    def __init__(self) :
        self.runnable = []
        self.timers = []
        self.cur = None
        self.running = True
    #end __init__

    def time(self) :
        return \
            time.monotonic()
    #end time

    def create_future(self) :
        "creates a Future which can be awaited until" \
        " it has a result."
        return \
            self.Future(self)
    #end create_future

    def create_task(self, coro) :
        "puts a coroutine object on the execution queue."
        result = self.Task(self, coro)
        self.runnable.append(result)
        return \
            result
    #end create_task

    def call_at(self, when, action, *actionargs) :
        return \
            self.Timer(self, when, action, actionargs)
    #end call_at

    def call_later(self, delay, action, *actionargs) :
        return \
            self.call_at(self.time() + delay, action, *actionargs)
    #end call_later

    def run_forever(self) :
        while self.running :
            try :
                self.cur = self.runnable.pop(0)
            except IndexError :
                self.cur = None
            #end try
            if self.cur != None :
                self.cur.resume()
                self.cur = None
            else :
                if len(self.timers) != 0 :
                    delay = self.timers[0].when - self.time()
                    if delay > 0 :
                        time.sleep(delay)
                        # event loop has nothing else to do
                    #end if
                    entry = heapq.heappop(self.timers)
                    entry.action(*entry.actionargs)
                else :
                    raise RuntimeError("no more tasks")
                #end if
            #end if
        #end while
    #end run_forever

    def stop(self) :
        self.running = False
    #end stop

#end CustomEventLoop

Some example code to make use of it:

In [ ]:
loop = CustomEventLoop()
timer_done = loop.create_future()

def set_timer_done() :
    timer_done.set_result(None)
#end set_timer_done

async def task1() :
    print("task1 starts")
    await timer_done
    print("timer_done is done, task1 finishes")
#end task1

async def task2() :
    print("task2 starts")
    await loop.create_task(task1())
    print("task1 done, task2 finishes")
    loop.stop()
#end task1

loop.call_later(2, set_timer_done)
loop.create_task(task2())
loop.run_forever()

This is fine, but there needs to be a more convenient way to suspend a coroutine: there needs to be a `CustomEventLoop.sleep()` method that they can call.

This can be done in a number of different ways. Here a `Future` is created, which will have a value put in it by a callback that is queued to run at the right time via `CustomEventLoop.call_later()`:

    def sleep(self, delay) :

        timer_done = self.create_future()

        def set_timer_done() :
            timer_done.set_result(None)
        #end set_timer_done

    #begin sleep
        self.call_later(delay, set_timer_done)
        return \
            timer_done
    #end sleep

Another way might be to make a `Timer` object itself directly awaitable, perhaps by associating it with a `Future` as is done with `Task` objects. Then the `sleep()` call can just return the timer object.

But anyway, here is the slightly-revised `CustomEventLoop` class:

In [ ]:
import time
import heapq

class CustomEventLoop :
    "event loop with timer callbacks and coroutine sleep() call."
    
    class Future :
        "something that a task can block on by awaiting, to be" \
        " resumed when another task puts a result into it."

        def __init__(self, loop) :
            self.loop = loop
            self.awaiting = []
            self.done = False
            self.result = None
        #end __init__

        def __await__(self) :
            if not self.done :
                self.awaiting.append(self.loop.cur)
                yield None # value yielded doesn’t matter
            #end if
            assert self.done
            return \
                self.result
        #end __await__

        def set_result(self, result) :
            "awakes all tasks awaiting this Future."
            assert not self.done
            self.done = True
            self.result = result
            for awaiting in self.awaiting :
                self.loop.runnable.append(awaiting)
            #end for
            self.awaiting = None
        #end set_result

    #end Future

    class Task :
        "schedulable wrapper for coroutine objects."

        def __init__(self, loop, coro) :
            self.coro = coro
            self.is_alive = True
            self.done = loop.create_future()
        #end __init__

        def __await__(self) :
            return \
                self.done.__await__()
        #end __await__

        def resume(self) :
            try :
                self.coro.send(None)
            except StopIteration :
                self.is_alive = False
                self.done.set_result(None)
            #end try
        #end resume

    #end Task

    class Timer :
        "for keeping track of timed callbacks."

        def __init__(self, loop, when, action, actionargs) :
            self.when = when
            self.loop = loop
            heapq.heappush(self.loop.timers, self)
            self.action = action
            self.actionargs = actionargs
        #end __init__

        def __lt__(a, b) :
            "defines enough ordering for heapq."
            return \
                a.when < b.when
        #end __lt__

    #end Timer
    
    def __init__(self) :
        self.runnable = []
        self.timers = []
        self.cur = None
        self.running = True
    #end __init__

    def time(self) :
        return \
            time.monotonic()
    #end time

    def create_future(self) :
        "creates a Future which can be awaited until" \
        " it has a result."
        return \
            self.Future(self)
    #end create_future

    def create_task(self, coro) :
        "puts a coroutine object on the execution queue."
        result = self.Task(self, coro)
        self.runnable.append(result)
        return \
            result
    #end create_task

    def call_at(self, when, action, *actionargs) :
        return \
            self.Timer(self, when, action, actionargs)
    #end call_at

    def call_later(self, delay, action, *actionargs) :
        return \
            self.call_at(self.time() + delay, action, *actionargs)
    #end call_later

    def sleep(self, delay) :

        timer_done = self.create_future()

        def set_timer_done() :
            timer_done.set_result(None)
        #end set_timer_done

    #begin sleep
        self.call_later(delay, set_timer_done)
        return \
            timer_done
    #end sleep
    
    def run_forever(self) :
        while self.running :
            try :
                self.cur = self.runnable.pop(0)
            except IndexError :
                self.cur = None
            #end try
            if self.cur != None :
                self.cur.resume()
                self.cur = None
            else :
                if len(self.timers) != 0 :
                    delay = self.timers[0].when - self.time()
                    if delay > 0 :
                        time.sleep(delay)
                        # event loop has nothing else to do
                    #end if
                    entry = heapq.heappop(self.timers)
                    entry.action(*entry.actionargs)
                else :
                    raise RuntimeError("no more tasks")
                #end if
            #end if
        #end while
    #end run_forever

    def stop(self) :
        self.running = False
    #end stop

#end CustomEventLoop

Example client code:

In [ ]:
loop = CustomEventLoop()

async def task1() :
    print("task1 starts")
    await loop.sleep(1)
    print("task1 did first sleep")
    await loop.sleep(0.5)
    print("task1 finishes")
#end task1

async def task2() :
    print("task2 starts")
    await loop.sleep(2)
    print("task2 finishes")
    loop.stop()
#end task2

loop.create_task(task1())
loop.create_task(task2())
loop.run_forever()

By the way, in my experience with coroutines in Python, it is very easy to forget to put the `await` on the front of the sleep calls. Then you wonder why your code keeps running without pausing to sleep!

## Further Enhancements ##

The [`asyncio.Future`](https://docs.python.org/3/library/asyncio-task.html#asyncio.Future) class allows cancellation, and also the setting of an exception to be raised in tasks waiting for a value. How would you implement this?

Similarly, the standard `call_at()` and `call_later()` methods return [Handle](https://docs.python.org/3/library/asyncio-eventloop.html#asyncio.Handle) objects, which can be cancelled to stop the timer before it goes off. How could this be done?

There are some other common needs that an event loop needs to support. An obvious one is [doing non-blocking I/O](https://docs.python.org/3/library/asyncio-eventloop.html#watch-file-descriptors): a task needs to be able to block until a file descriptor becomes readable or writable, without holding up other tasks. Obviously, the event loop can use one of the [select/poll](https://docs.python.org/3/library/select.html) calls to implement this.

The details of all these will be left as an exercise for the reader. 😉


## Conclusion ##

Event loops are already commonplace in programming -- every GUI toolkit provides one, for example. But there is no standard API for them. Python’s `asyncio` is, I think a unique development in the programming world, by trying to provide such a standard API. This is more important than the fact that `asyncio` implements a full-function event loop of its own, because after all those GUI toolkits are not going to give up their existing event-loop implementations.

Instead, all you need to do is provide a wrapper for the existing event loop, in the form of a compatible subclass of [`asyncio.AbstractEventLoop`](https://docs.python.org/3/library/asyncio-eventloop.html#asyncio.AbstractEventLoop). Then it becomes possible to write “event-loop-agnostic” code, that will work unchanged with any existing event loop!
